In [1]:
import math
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import string
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance

import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [11]:
train_text =np.load('Mal_train.npy')
print(train_text.shape)
train_text = train_text.tolist()
#print(train_text)
train_label =np.load('Mal_train_label.npy')
print(train_label.shape)
train_label = train_label.tolist()
#print(train_label)

(1000,)
(1000,)


In [9]:
valid_text =np.load('Mal_val.npy')
print(valid_text.shape)
valid_text = valid_text.tolist()
# print(valid_text)
valid_label =np.load('Mal_val_label.npy')
print(valid_label.shape)
valid_label = valid_label.tolist()
# print(valid_label)

(100,)
(100,)


In [12]:
# train_text = train_df['text'].tolist()
# valid_text = valid_df['text'].tolist()
# test_text = test_df['text'].tolist()

# train_label = train_df['category'].tolist()
# valid_label = valid_df['category'].tolist()

In [13]:
#function definition
def get_model(model_url, max_seq_length):
  inputs = dict(
    input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    ) 
  muril_layer = hub.KerasLayer(model_url, trainable=True)
  outputs = muril_layer(inputs)
  assert 'sequence_output' in outputs
  assert 'pooled_output' in outputs
  assert 'encoder_outputs' in outputs
  assert 'default' in outputs
  return tf.keras.Model(inputs=inputs,outputs=outputs["pooled_output"]), muril_layer
#function call
max_seq_length = 128
muril_model, muril_layer = get_model(
    model_url="https://tfhub.dev/google/MuRIL/1", max_seq_length=max_seq_length)

In [14]:
#converts input into BERT-acceptable format (preprocessing)
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [15]:
#That 3 types of embeddings in BERT
def create_input(input_strings, tokenizer, max_seq_length):
  input_ids_all, input_mask_all, input_type_ids_all = [], [], []
  for input_string in input_strings:
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))
    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)
    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    input_type_ids_all.append([0] * max_seq_length)
  return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [16]:
def encode(input_text):
  input_ids, input_mask, input_type_ids = create_input(input_text,tokenizer,max_seq_length)
  inputs = dict(
      input_word_ids=input_ids,
      input_mask=input_mask,
      input_type_ids=input_type_ids,
  )
  return muril_model(inputs)

In [18]:
import math
import time

In [19]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(train_text)/Batch_size)

In [20]:
train_text[0:10]

['dislike  adich madkununindal9 mohanlal fanz',
 'puli murukan  ithokke enth',
 'ഇതിലും ഫേധം തെലുഗ് ഡബ്ബിങ് സിനിമകളിലെ പാട്ടുകൾ  ആണ് എന്ന് തോന്നി പോയ നിമിഷം ',
 '120 muthalulla bgm kettappol sacred games orma vannavarundo',
 'aval aa kadele freezer il alle  ithipo kadha motham ondallo trailer il',
 'sambavam nml kandu madutha story aanu telugu and tamil same cliche but namuk idh pernal avum kidu trailer',
 'newton enna hindi cinema orma varunnu',
 'powlichu eth vere level movie ayirikum',
 'സ്കൂള്പഇലെർക് പറ്റിയ ട്രൈലെർ പോയിനെടാ എടുത്തോണ്ട് നിത്യ ഗർഭിണി ഫാൻസ്\u200c',
 'trailer crisp akaamayirunnutoo much aayi poyi']

In [21]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(train_text)
        out = encode(train_text[int(start):int(Endu)])
    else:
        out = encode(train_text[int(start):int(Endu)])
    if i == 0:
        total = out
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [22]:
X_train = np.array(total)
np.save('Muril_X_train.npy', X_train)

In [23]:
X_train = np.load('Muril_X_train.npy')

In [24]:
X_train.shape

(1000, 768)

In [25]:
Y_train = np.array(train_label)

In [27]:
Y_train.shape

(1000,)

In [28]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(valid_text)/Batch_size)

In [29]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(valid_text)
        out = encode(valid_text[int(start):int(Endu)])
    else:
        out = encode(valid_text[int(start):int(Endu)])
    if i == 0:
        total = out
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0


In [30]:
X_val = np.array(total)
np.save('Muril_X_val.npy', X_val)

In [31]:
X_val = np.load('Muril_X_val.npy')

In [32]:
Y_val = np.array(valid_label)

# SVM

In [33]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [34]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(decision_function_shape='ovo')

In [35]:
pred_svc = svc.predict(X_val)

In [36]:
confusion_matrix(Y_val,pred_svc)

array([[10,  0,  5,  5,  0],
       [11,  0,  8,  1,  0],
       [ 7,  0,  8,  5,  0],
       [ 1,  0,  0, 19,  0],
       [ 7,  0,  6,  7,  0]], dtype=int64)

In [37]:
print(classification_report(Y_val,pred_svc))

              precision    recall  f1-score   support

           0       0.28      0.50      0.36        20
           1       0.00      0.00      0.00        20
           2       0.30      0.40      0.34        20
           3       0.51      0.95      0.67        20
           4       0.00      0.00      0.00        20

    accuracy                           0.37       100
   macro avg       0.22      0.37      0.27       100
weighted avg       0.22      0.37      0.27       100



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## DecisionTree

In [38]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [40]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [41]:
pred_Dtree = Dtree.predict(X_val)

In [42]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 5,  4,  6,  0,  5],
       [ 6,  5,  5,  0,  4],
       [ 2,  4,  4,  2,  8],
       [ 3,  1,  3, 12,  1],
       [ 4,  5,  5,  2,  4]], dtype=int64)

In [43]:
print(classification_report(Y_val,pred_Dtree))

              precision    recall  f1-score   support

           0       0.25      0.25      0.25        20
           1       0.26      0.25      0.26        20
           2       0.17      0.20      0.19        20
           3       0.75      0.60      0.67        20
           4       0.18      0.20      0.19        20

    accuracy                           0.30       100
   macro avg       0.32      0.30      0.31       100
weighted avg       0.32      0.30      0.31       100



## Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier()

In [46]:
pred_RFC = RFC.predict(X_val)

In [47]:
confusion_matrix(Y_val,pred_RFC)

array([[ 7,  4,  4,  4,  1],
       [ 4, 11,  4,  0,  1],
       [ 5,  4,  5,  2,  4],
       [ 2,  1,  0, 15,  2],
       [ 6,  5,  4,  2,  3]], dtype=int64)

In [48]:
print(classification_report(Y_val,pred_RFC))

              precision    recall  f1-score   support

           0       0.29      0.35      0.32        20
           1       0.44      0.55      0.49        20
           2       0.29      0.25      0.27        20
           3       0.65      0.75      0.70        20
           4       0.27      0.15      0.19        20

    accuracy                           0.41       100
   macro avg       0.39      0.41      0.39       100
weighted avg       0.39      0.41      0.39       100



# Denoisng

In [1]:
import cupy as cp

In [2]:
X_train = cp.load('Muril_X_train.npy')
X_val = cp.load('Muril_X_val.npy')
Y_train = cp.load('Mal_train_label.npy')
Y_val = cp.load('Mal_val_label.npy')

In [3]:
from WR_LS_Denoising import WR_LS_Denoise

In [4]:
import time

In [5]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_train)):
    Denoised =  WR_LS_Denoise(X_train[i])
    X_train[i] = X_train[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Sat Oct 23 09:44:31 2021
0 Sat Oct 23 09:44:33 2021


In [6]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_val)):
    Denoised =  WR_LS_Denoise(X_val[i])
    X_val[i] = X_val[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Sat Oct 23 09:45:20 2021
0 Sat Oct 23 09:45:20 2021


In [7]:
WR_LS_Denoised_X_train = cp.save('WR_LS_Denoised_MuRIL_X_train.npy',X_train)
WR_LS_Denoised_X_val = cp.save('WR_LS_Denoised_MuRIL_X_val.npy',X_val)

In [9]:
import numpy as np

In [10]:
X_train = np.load('WR_LS_Denoised_MuRIL_X_train.npy')
X_val = np.load('WR_LS_Denoised_MuRIL_X_val.npy')

In [11]:
Y_train = np.load('Mal_train_label.npy')
Y_val = np.load('Mal_val_label.npy')

# SVM

In [12]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(decision_function_shape='ovo')

In [14]:
pred_svc = svc.predict(X_val)

In [15]:
confusion_matrix(Y_val,pred_svc)

array([[10,  0,  5,  5,  0],
       [11,  0,  8,  1,  0],
       [ 7,  0,  8,  5,  0],
       [ 1,  0,  0, 19,  0],
       [ 7,  0,  6,  7,  0]], dtype=int64)

In [16]:
print(classification_report(Y_val,pred_svc))

              precision    recall  f1-score   support

           0       0.28      0.50      0.36        20
           1       0.00      0.00      0.00        20
           2       0.30      0.40      0.34        20
           3       0.51      0.95      0.67        20
           4       0.00      0.00      0.00        20

    accuracy                           0.37       100
   macro avg       0.22      0.37      0.27       100
weighted avg       0.22      0.37      0.27       100



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Decision Tree

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [22]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [23]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [24]:
pred_Dtree = Dtree.predict(X_val)

In [25]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 5,  4,  6,  0,  5],
       [ 6,  5,  5,  0,  4],
       [ 2,  4,  4,  2,  8],
       [ 3,  1,  3, 12,  1],
       [ 4,  5,  5,  2,  4]], dtype=int64)

In [26]:
print(classification_report(Y_val,pred_Dtree))

              precision    recall  f1-score   support

           0       0.25      0.25      0.25        20
           1       0.26      0.25      0.26        20
           2       0.17      0.20      0.19        20
           3       0.75      0.60      0.67        20
           4       0.18      0.20      0.19        20

    accuracy                           0.30       100
   macro avg       0.32      0.30      0.31       100
weighted avg       0.32      0.30      0.31       100



# Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier()

In [19]:
pred_RFC = RFC.predict(X_val)

In [20]:
print(classification_report(Y_val,pred_RFC))

              precision    recall  f1-score   support

           0       0.41      0.45      0.43        20
           1       0.42      0.50      0.45        20
           2       0.46      0.30      0.36        20
           3       0.71      0.85      0.77        20
           4       0.47      0.40      0.43        20

    accuracy                           0.50       100
   macro avg       0.49      0.50      0.49       100
weighted avg       0.49      0.50      0.49       100

